In [1]:
import datajoint as dj
import numpy as np
import time

from meshparty import trimesh_io

In [12]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


In [17]:
#ta3p100.FilteredSkeleton()
ta3p100.NeuriteRawSkeletonSpan()
ta3p100.NeuriteRawSkeletonReach().drop()

`microns_ta3p100`.`__neurite_raw_skeleton_reach` (32929 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [4]:
# #how to extract the bounding box

# key = dict(segmentation=2,segment_id=648518346341351731)
# skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
# print(skeleton_edges.shape)
# vertices = skeleton_edges.reshape(-1,3)
# vertices
# mins = np.amin(vertices, axis=0)
# maxs = np.amax(vertices, axis=0)
# print("mins,maxs = " + str((mins,maxs)))
# diff = maxs-mins
# volume = diff[0]*diff[1]*diff[2]
# print("volume = " + str(volume/1000000000))

In [5]:
# #how to extract the lengths
# filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
# sum(filtered_edges_distance_lin_alg)

In [6]:
#ta3p100.Synapse.describe()

In [7]:
@schema
class NeuriteRawSkeletonSpan(dj.Computed):
    definition="""
    -> ta3p100.FilteredSkeleton
    ---
    bbox_volume   :double unsigned #length of the entire skeleton, raw_volume/10^9
    skeleton_length    :double unsigned #length of the entire skeleton
    skel_bbox_x1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_y1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_z1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_x2          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_y2          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_z2          : bigint unsigned              # (EM voxels)  - bounding box
    """
    
    def make(self, key):
        start_time = time.time()
        print(str(key["segment_id"]) + ":")
        #get the skeleton edges
        skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
        vertices = skeleton_edges.reshape(-1,3)
        
        mins = np.amin(vertices, axis=0)
        maxs = np.amax(vertices, axis=0)
        #print("mins,maxs = " + str((mins,maxs)))
        #key["bbox_corners"] = (mins,maxs)
        key["skel_bbox_x1"] = mins[0]
        key["skel_bbox_y1"] = mins[1]
        key["skel_bbox_z1"] = mins[2]
        key["skel_bbox_x2"] = maxs[0]
        key["skel_bbox_y2"] = maxs[1]
        key["skel_bbox_z2"] = maxs[2]

        
        diff = maxs-mins
        volume = diff[0]*diff[1]*diff[2]
        #print("volume = " + str(volume/1000000000))
        key["bbox_volume"] = volume/1000000000
        
        filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
        total_length = sum(filtered_edges_distance_lin_alg)
        
        key["skeleton_length"] = total_length
        
        self.insert1(key,skip_duplicates=True)
        
        print(f"Total Time: {time.time()-start_time}")
        print()
        
    
    

In [8]:
#ta3p100.NeuriteRawSkeletonSpan.drop()

In [9]:
#ta3p100.NeuriteRawSkeletonReach.drop()

In [10]:
#(schema.jobs & "table_name='__neurite_raw_skeleton_span'").delete()

In [11]:
start_time = time.time()
NeuriteRawSkeletonSpan.populate(reserve_jobs=True)
print(f"Total time for populate: {time.time()-start_time}")

648518346341351467:
Total Time: 0.02664971351623535

648518346341351503:
Total Time: 0.022885799407958984

648518346341351508:
Total Time: 0.025821924209594727

648518346341351509:
Total Time: 0.022560834884643555

648518346341351512:
Total Time: 0.026711463928222656

648518346341351514:
Total Time: 0.02320718765258789

648518346341351518:
Total Time: 0.026381492614746094

648518346341351523:
Total Time: 0.037848472595214844

648518346341351525:
Total Time: 0.020425081253051758

648518346341351531:
Total Time: 0.030192136764526367

648518346341351545:
Total Time: 0.015636682510375977

648518346341351631:
Total Time: 0.03801727294921875

648518346341351638:
Total Time: 0.019379615783691406

648518346341351695:
Total Time: 0.019622325897216797

648518346341351710:
Total Time: 0.026674747467041016

648518346341351713:
Total Time: 0.030059099197387695

648518346341351718:
Total Time: 0.03848457336425781

648518346341351719:
Total Time: 0.016029834747314453

648518346341351722:
Total Time: 

OperationalError: (1213, 'Deadlock found when trying to get lock; try restarting transaction')